In [21]:
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine, Column, Integer, Date
Base = declarative_base()
# 모델 정의
class Lotto(Base):
    __tablename__ = 'lotto'
    turn = Column(Integer, primary_key=True) # 회차
    date = Column(Date) # 추첨일
    grade1count = Column(Integer) # 1등 당첨자수
    grade1money = Column(Integer) # 1등 당첨금액
    grade2count = Column(Integer)
    grade2money = Column(Integer)
    grade3count = Column(Integer)
    grade3money = Column(Integer)
    grade4count = Column(Integer)
    grade4money = Column(Integer)
    grade5count = Column(Integer) # 5등
    grade5money = Column(Integer) # 5등
    win1 = Column(Integer) # 당첨번호 첫번째
    win2 = Column(Integer) # 당첨번호 두번째
    win3 = Column(Integer) # 당첨번호 세번째
    win4 = Column(Integer) # 당첨번호 네번째
    win5 = Column(Integer) # 당첨번호 다섯번째
    win6 = Column(Integer) # 당첨번호 여섯번째
    winb = Column(Integer) # 당첨번호 보너스

In [32]:
import requests #pip install requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
from bs4 import NavigableString
import datetime
# 신규 당첨 크롤링
def newWinCrawling():
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'}
    url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
    response = requests.get(url, headers=headers) 
    html_content = response.text
    # Beautiful Soup을 사용하여 HTML 내용 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    
    new_win = Lotto()
    
    turn = soup.find('div', class_="win_result").find('h4').find('strong').get_text() #회차
    turn = int(turn.replace('회', '').strip())
    new_win.turn = int(turn)

    date = soup.find('p', class_='desc')
    print('date : ', date.get_text().replace('(','').replace(')', ''))
    date_arr = date.get_text().replace('(','').replace(')', '').split()
    print(date_arr)
    new_win.date = datetime.date(int(date_arr[0].replace('년', '')), int(date_arr[1].replace('월', '')), int(date_arr[2].replace('일', '')))

    win_num_span = soup.find('div', class_="win_result").find('span')
    bonus = int(soup.find('div', class_="num bonus").find('span').get_text())
    new_win.winb = int(bonus)
    
    win_list = []
    while len(win_list) < 6 :
        if win_num_span :
            win_list.append(int(win_num_span.get_text()))
        else : 
            break
        win_num_span = win_num_span.find_next_sibling()
    new_win.win1 = win_list[0]
    new_win.win2 = win_list[1]
    new_win.win3 = win_list[2]
    new_win.win4 = win_list[3]
    new_win.win5 = win_list[4]
    new_win.win6 = win_list[5]
    win_list.insert(0, turn)
    win_list.append(bonus)

    # ret = soup.find('tbody').find('tr')
    count_arr = []
    money_arr = []
    tr_arr : NavigableString =  soup.find('tbody').find_all('tr')
    for i, v in enumerate(tr_arr):
        v : NavigableString = v.find_all('td')
        # print(v)
        for j, k in enumerate(v):
            k : NavigableString = k
            if j == 2: # 당첨자수
                print(i, k.get_text())
                count_arr.append(int(k.get_text().replace(',', '')))
            elif j == 3: # 당첨금액
                print(i, k.get_text())
                money_arr.append(int(k.get_text().replace('원', '').replace(',', '')))

    new_win.grade1count = count_arr[0]
    new_win.grade2count = count_arr[1]
    new_win.grade3count = count_arr[2]
    new_win.grade4count = count_arr[3]
    new_win.grade5count = count_arr[4]
    
    new_win.grade1money = money_arr[0]
    new_win.grade2money = money_arr[1]
    new_win.grade3money = money_arr[2]
    new_win.grade4money = money_arr[3]
    new_win.grade5money = money_arr[4]
    

    # print(len(soup.find('tbody').find_all('tr')))
    # # print(ret.find_all('td'))
    # for i, v in enumerate(ret.find_all('td')):
    #     if i == 2: # 당첨자수
    #         print(v.get_text())
    #     elif i == 3: # 당첨금액
    #         print(v.get_text())
    # print(ret)
    # print(ret.find_next_sibling())
    print(f'turn {new_win.turn} \
    date {new_win.date} \
    grade1count {new_win.grade1count} \
    grade1money {new_win.grade1money} \
    grade2count {new_win.grade2count} \
    grade2money {new_win.grade2money} \
    grade3count {new_win.grade3count} \
    grade3money {new_win.grade3money} \
    grade4count {new_win.grade4count} \
    grade4money {new_win.grade4money} \
    grade5count {new_win.grade5count} \
    grade5money {new_win.grade5money} \
    win1 = {new_win.win1} \
    win2 = {new_win.win2} \
    win3 = {new_win.win3} \
    win4 = {new_win.win4} \
    win5 = {new_win.win5} \
    win6 = {new_win.win6} \
    winb = {new_win.winb}')
    return win_list

newWinCrawling()

date :  2024년 09월 07일 추첨
['2024년', '09월', '07일', '추첨']
0 12
0 2,314,468,157원
1 75
1 61,719,151원
2 2,906
2 1,592,890원
3 144,451
3 50,000원
4 2,452,383
4 5,000원
turn 1136     date 2024-09-07     grade1count 12     grade1money 2314468157     grade2count 75     grade2money 61719151     grade3count 2906     grade3money 1592890     grade4count 144451     grade4money 50000     grade5count 2452383     grade5money 5000     win1 = 21     win2 = 33     win3 = 35     win4 = 38     win5 = 42     win6 = 44     winb = 1


[1136, 21, 33, 35, 38, 42, 44, 1]

[1136, 21, 33, 35, 38, 42, 44, 1]